In [5]:
%pip install kaggle

  Using cached kaggle-1.7.4.5-py3-none-any.whl.metadata (16 kB)
  Using cached bleach-6.2.0-py3-none-any.whl.metadata (30 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached charset_normalizer-3.4.2-cp312-cp312-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached protobuf-6.31.0-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached python_slugify-8.0.4-py2.py3-none-any.whl.metadata (8.5 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached setuptools-80.7.1-py3-none-any.whl.metadata (6.6 kB)
Using cached kaggle-1.7.4.5-py3-none-any.whl (181 kB)
Using cached certifi-2025.4.26-py3-none-any.whl (159 kB)
Using cached setuptools-80.7.1-py3-none-any.whl (1.2 MB)
Using cached bleach-6.2.0-py3-none-any.whl (163 kB)
Using cached charset_normalizer-3.4.2-cp312-cp312-win_amd64.whl (105 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached protobuf-6.31.0-cp310-ab

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Movie Project\\.venv\\Lib\\site-packages\\setuptools\\_distutils\\tests\\test_bdist_rpm.py'
Check the permissions.



In [7]:
!kaggle datasets list -s "movie"

ref                                                             title                                            size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -----------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
danielgrijalvas/movies                                          Movie Industry                                 433664  2021-07-23 19:48:26.173000          57266        455  1.0              
tmdb/tmdb-movie-metadata                                        TMDB 5000 Movie Dataset                       9317430  2017-09-28 01:09:12.897000         433913       4009  0.8235294        
jrobischon/wikipedia-movie-plots                                Wikipedia Movie Plots                        31376869  2018-10-15 19:59:54.650000          28871        505  0.88235295       
shivamb/netflix-shows                        

In [8]:
!kaggle datasets download tmdb/tmdb-movie-metadata

Dataset URL: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata
License(s): other




  0%|          | 0.00/8.89M [00:00<?, ?B/s]
100%|██████████| 8.89M/8.89M [00:00<00:00, 462MB/s]


 Unzip the file to access contents and list files

In [10]:
import zipfile
import os

with zipfile.ZipFile("tmdb-movie-metadata.zip", "r") as zip_ref:
    zip_ref.extractall("tmdb_movie_data")

os.listdir("tmdb_movie_data")

['tmdb_5000_credits.csv', 'tmdb_5000_movies.csv']

Read CSV files into pandas dataframe